In [1]:
# import modules
import glob
import json
import math
import numpy as np
import pandas as pd
import re

In [2]:
# load the json files into a dataframe
pose_keypoints = []

json_files = glob.glob('output_jsons/*.json')

for filename in json_files:
    print("Reading: {}".format(filename))
    with open(filename, 'r') as file:
        data = json.loads(file.read())
        
    df = pd.json_normalize(data, record_path=['people'])
    pose_keypoints.append(pd.Series(df.pose_keypoints_2d[0]))
    
pose_keypoints = pd.concat(pose_keypoints, axis=1)
pose_keypoints = pose_keypoints.T

Reading: output_jsons\CMJ_keypoints.json
Reading: output_jsons\DJ_keypoints.json
Reading: output_jsons\SL_CMJ_L_keypoints.json


In [3]:
# add columns names
pose_keypoints.columns = [x + str(y) for y in range(25) for x in ['x', 'y', 'c']]

In [4]:
# isolate the required body parts
l_hip = pose_keypoints[['x12', 'y12']]
l_knee = pose_keypoints[['x13', 'y13']]
l_ankle = pose_keypoints[['x14', 'y14']]

In [5]:
# create a function that calculates the angle between three points
def angle_calc(p0, p1, p2):
    try:
        a = (p1[0] - p0[0])**2 + (p1[1] - p0[1])**2
        b = (p1[0] - p2[0])**2 + (p1[1] - p2[1])**2
        c = (p2[0] - p0[0])**2 + (p2[1] - p0[1])**2
        angle = math.acos((a + b - c) / math.sqrt(4 * a * b)) * 180 / math.pi
    except:
        return 0
    return round(angle, 4)

In [6]:
# calculate the angle between left hip, knee, and ankle
for i, filename in enumerate(json_files):
    angle = angle_calc(l_hip.iloc[i, :], l_knee.iloc[i, :], l_ankle.iloc[i, :])
    print(re.sub('[^A-Z]', '', filename), "left knee angle:", angle, "degrees")

CMJ left knee angle: 92.969 degrees
DJ left knee angle: 99.9884 degrees
SLCMJL left knee angle: 106.6489 degrees
